In [ ]:
%load_ext autoreload
%autoreload 2
from dataloader import *
from training_utils import *
import torch
import os
import pickle
from torch.utils.data.dataset import random_split

torch.set_num_threads(os.cpu_count())

trashset = TrashNetDataset("trashnet/data/dataset-resized")
valsize = int(len(trashset) * 0.1)
trainsize = int(len(trashset) * 0.8)
testsize = len(trashset) - valsize - trainsize
torch.manual_seed(0) # Ensure dataset is randomly split the same way each time
train_dataset, val_dataset, test_dataset = random_split(trashset, [trainsize, valsize, testsize])

train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

Loading PyTorch .checkpoint model and plotting training metadata. Make sure to set the loadedModel variable to have the same architecture as the initial trained model. For example, if the initial trained model was ResNet152, initialize loadedModel as `loadedModel = resnet152(pretrained=False)`. Also remember to set the model name so it matches the saved model.

In [ ]:
from torchvision.models.resnet import *

modelName = "test"
loadedModel = resnet18(pretrained=False)
loadedModel.fc = torch.nn.Linear(loadedModel.fc.in_features, len(CLASSES), bias=True)
loadedModel.load_state_dict(torch.load("models/" + modelName + "/weights.checkpoint"))
loadedModel.eval()
for param in loadedModel.parameters():
    param.requires_grad = False
loadedModel.device = 'cpu'
if torch.cuda.is_available():
    loadedModel.device = 'cuda'
loadedModel.to(loadedModel.device)
acc = evaluate(loadedModel, test_dataloader)
print("Accuracy:", acc)
pickle_in = open("models/" + modelName + "/training_metadata.pickle","rb")
loaded_obj = pickle.load(pickle_in)
pickle_in.close()
total_loss = loaded_obj["total_loss"]
total_acc = loaded_obj["total_acc"]
total_learning_rate = loaded_obj["total_learning_rate"]
total_batch_size = loaded_obj["total_batch_size"]
total_train_full = loaded_obj["total_train_full"]
num_epochs = loaded_obj["num_epochs"]

fig = plt.figure(figsize=(20,3))
plt.subplot(141)
plt.plot(total_loss)
plt.xlabel("Iteration")
plt.ylabel("Training Loss")
plt.subplot(142)
plt.plot(total_acc)
plt.xlabel("Iteration")
plt.ylabel("Validation Accuracy")
plt.subplot(143)
plt.plot(total_batch_size)
plt.xlabel("Iteration")
plt.ylabel("Batch Size")
plt.subplot(144)
plt.plot(torch.log10(torch.tensor(total_learning_rate)))
plt.xlabel("Iteration")
plt.ylabel("Learning Rate (log10)")
plt.show()

Creating ScriptModule to export to C++.

In [ ]:
transcribedModel = resnet18(pretrained=False)
transcribedModel.fc = torch.nn.Linear(transcribedModel.fc.in_features, len(CLASSES), bias=True)
transcribedModel.load_state_dict(torch.load("models/" + modelName + "/weights.checkpoint"))
transcribedModel.eval()
for param in transcribedModel.parameters():
    param.requires_grad = False

x, _ = trashset[0]
traced_script_module = torch.jit.trace(transcribedModel, x.unsqueeze(0))

transcribedModel.device = 'cpu'
traced_script_module.device = 'cpu'
traced_script_module.save("models/" + modelName + "/transcripted_model_cpu.pt")
if torch.cuda.is_available():
    transcribedModel.device = 'cuda'
    traced_script_module.device = 'cuda'
transcribedModel.to(transcribedModel.device)
traced_script_module.to(traced_script_module.device)

if torch.cuda.is_available():
    traced_script_module.save("models/" + modelName + "/transcripted_model_cuda.pt")

Verify training accuracy (running is optional).

In [ ]:
accScipted = evaluate(traced_script_module, test_dataloader)
print("Scripted Model Accuracy:", accScipted)
accExpected = evaluate(transcribedModel, test_dataloader)
print("Expected Model Accuracy:", accExpected)